![image.png](attachment:image.png)

## PRACTICA OBLIGATORIA: **Redes Convolucionales**

* La práctica obligatoria de esta unidad consiste en un ejercicio de construcción de una red convolucional con Keras para la clasificación de imagenes de simpáticos perretes y gatetes. Descarga este notebook en tu ordenador y trabaja en local. Ten en cuenta que tendrás que descar los directorios de imágenes y datos adicionales, si los hubiera.
* Recuerda que debes subirla a tu repositorio personal antes de la sesión en vivo para que puntúe adecuadamente.  
* Recuerda también que no es necesario que esté perfecta, sólo es necesario que se vea el esfuerzo.
* Esta práctica se resolverá en la sesión en vivo correspondiente y la solución se publicará en el repo del curso.

### Ejercicio 0

Importa los paquetes y módulos que necesites a lo largo del notebook.

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from skimage.io import imread
from sklearn.utils import shuffle
import seaborn as sns
from sklearn.metrics import confusion_matrix
import keras
import cv2
from google.colab import files

### Problema de Negocio y dataset

Una conocida empresa de información inmobiliaria utiliza un CAPTCHA visual de perros y gatos para detectar webscrappings intensivos y ataques de denegación de servicio. Últimamente, ha detectado que su sistema está empezando a flojear, es decir que hay muchos sistemas de detección automática de imágenes que son capaces de detectar que es un perro y un gato y salterse el CAPTCHA antirobots. Por eso nos ha pedido que entrenemos un modelo potente de clasificación que emule a estos sistemas con el objetivo de emplear como fotografías para el CAPTCHA aquellas que no sepa detectar este sistema emulador de chicos malos.  

Pues manos a la obra, tenemos que seleccionar en un conjunto de 1000 imágenes aquellas que peor se le daría clasificar a un potencial sistema anti-CAPTCHA.  

Para ello vamos a emplear un conjunto de train de 4000 imágenes de perretes y gatetes que se encuentran distribuidas en "data" en cuatro directorios "github_train_0",  "github_train_1", "githun_train_2" y "github_train_3". Los datos de las imagenes que tenemos que clasificar y luego seleccionar como las más "difíciles" están en "data" en el directorio "github_test".


### Se pide

1. Crear los datasets X,y de train y test, leyendo las imágenes de los directorios correspondientes (tendrás que leer todas las imágenes de los cuatro directorios para train) y a la vez que se leen adpatar su resolución para que todas tengan un tamaño de 32x32. En este caso puedes adaptar las funciones de los ejercicios de la unidad. NOTA: Ten en cuenta que la clase a la que pertenece en la foto está en el nombre del archivo.

2. Crea un modelo DL basado en redes convolucionales con al menos dos juegos Convolucional-Pooling. Sigue todo el proceso de ML (visualización del dataset, MiniEDA, Construcción del modelo)

3. Entrenar el modelo con un callback de Earlystopping con paciencia la que tengas :-). Muestra su historial de entrenamiento.

4. Evalua el modelo, haz un reporte de clasificacion y muestra la matriz de confianza.

5. Es el momento de seleccionar las imagenes: Selecciona el 10% de imágenes mal clasificadas de una y otra clase que tengan el mayor nivel de confianza de que pertenencían a la clase errónea. Es decir las imágenes de perros clasificadas como gatos y con la probabilidad de ser perror más alta.


### Extra (para hacer en clase)

Repite los pasos 3 a 5 utilizando el generador de imágenes sintéticas o Image Augmentation de Keras.



In [9]:
uploaded_0 = files.upload()

In [ ]:
import zipfile
import io
data= zipfile.ZipFile(io.BytesIO(uploaded_0['data_zip.zip']), 'r')
data.extractall()

In [27]:
#Leemos las rutas y añadimos a una lista todas las fotos

ruta_train_0 ="/content/data_zip/github_train_0"
ruta_train_1 ="/content/data_zip/github_train_1"
ruta_train_2 ="/content/data_zip/github_train_2"
ruta_train_3 ="/content/data_zip/github_train_3"
rutas_train = [ruta_train_0,ruta_train_1, ruta_train_2,ruta_train_3]
img_size = 32
lista_train = []
lista_train_y = []

for ruta in rutas_train:
  for img_name in os.listdir(ruta):
    #Ruta
    img_path = os.path.join(ruta, img_name)
    #Leer imagen
    img = cv2.imread(img_path)
    #Pasar a gris
    img_grey =cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #Redimensionar imagen
    img_grey_resize = cv2.resize(img_grey,(img_size,img_size))
    #Añadir a la lista
    lista_train.append([img_grey_resize])
    #Obtener nombre del archivo
    label = os.path.splitext(img_name)[0]
    lista_train_y.append(label)

print(len(lista_train), len(lista_train_y))

4000 4000


In [28]:
#Convertimos la lista en una matriz
array_train = np.array(lista_train)
array_train.shape
array_train_y = np.array(lista_train_y)
array_train_y.shape

(4000,)

In [29]:
#Aplanamos la matriz
flatten_train = array_train.reshape(array_train.shape[0],-1)
flatten_train.shape
flatten_train_y = array_train_y.reshape(array_train_y.shape[0],-1)
flatten_train_y.shape

(4000, 1)

In [30]:
#Obtenemos el dataframe de train
X_train = pd.DataFrame(flatten_train)
X_train
y_train = pd.DataFrame(flatten_train_y)
y_train

,0
0,cat.424
1,cat.10037
2,cat.743
3,cat.1226
4,cat.3406
...,...
3995,dog.11918
3996,dog.10222
3997,dog.11139
3998,dog.12104


In [39]:
#Repetimos proceso para el test

ruta_test ="/content/data_zip/github_test"
img_size = 32
lista_test = []
lista_test_y = []

for img_name in os.listdir(ruta_test):
  #Ruta
  img_path = os.path.join(ruta_test, img_name)
  #Leer imagen
  img=cv2.imread(img_path)
  #Pasar a gris
  img_grey =cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  #Redimensionar
  img_grey_resize = cv2.resize(img_grey,(img_size,img_size))
  #Aañadir a la lista
  lista_test.append([img_grey_resize])
  #Obtener nombre del archivo
  label = os.path.splitext(img_name)[0]
  lista_test_y.append(label)

print(len(lista_test), len(lista_test_y))

1000 1000


In [47]:
#Convertimos la lista en una matriz
array_test = np.array(lista_test)
array_test.shape
array_test_y = np.array(lista_test_y)
array_test_y.shape

(1000,)

In [50]:
#Aplanamos la matriz
flatten_test = array_test.reshape(array_test.shape[0],-1)
flatten_test.shape
flatten_test_y = array_test_y.reshape(array_test_y.shape[0],-1)
flatten_test_y.shape

(1000, 1)

In [52]:
#Obtenemos el dataframe de train
X_test = pd.DataFrame(flatten_test)
X_test
y_test = pd.DataFrame(flatten_test_y)
y_test

,0
0,cat.11717
1,dog.9621
2,cat.11652
3,cat.11878
4,dog.8602
...,...
995,cat.11781
996,cat.11441
997,cat.12228
998,cat.12051


In [53]:
X_train.head(2)

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,228,230,229,232,227,224,211,204,214,128,...,240,242,244,245,242,238,240,236,232,223
1,158,161,161,175,145,92,110,157,156,141,...,24,27,27,15,16,24,18,60,92,102


In [54]:
y_train.head(2)

,0
0,cat.424
1,cat.10037


In [55]:
X_test.head(2)

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,20,17,22,19,27,29,28,43,52,48,...,159,162,158,150,161,171,155,145,150,139
1,124,130,128,131,131,132,134,135,136,137,...,155,169,167,149,150,146,134,163,161,156


In [57]:
y_test.head(2)

,0
0,cat.11717
1,dog.9621
